# Understanding the NestedFrame

## Learning Objectives

By the end of this tutorial, you will:

* Learn about general capabilities of the DataFrame-like API that LSDB is built on top of, the NestedFrame.
* Understand common methods to package data as nested columns
* Understand common approaches to working with nested columns

## Introduction

The LSDB catalog is built on top of the pandas/dask ecosystem, where much of the look and feel of a pandas DataFrame is present in catalog operations. Pandas DataFrames are fairly ubiquitous, both in astronomy-specific applications and the broader scientific python community. If you haven't encountered them before, you can read about them in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/10min.html). 

However, in complex and interconnected astronomical datasets, the pandas DataFrame has several limitations. The [nested-pandas](https://nested-pandas.readthedocs.io/en/latest/) package was developed with these specific limitations in mind, and provides a performant extension class, the NestedFrame. The main idea of the NestedFrame, is to provide the ability to nest DataFrames within DataFrames, an example being in time-domain astronomy where the photometry table corresponding to an object table can be directly nested within a column of the object table.

The core advantages being:
* **No Duplication of Data**: In native pandas one might be tempted to simply join the tables, but this will duplicate all columns from the object table. With many object table columns, this can severely increase the memory footprint of the full dataset.
* **Avoiding Grouping Operations**: Nested Datasets are pre-grouped by the top-level index, avoiding grouping operations that come with storing these datasets as flat tables (for example, when doing operations that apply to a lightcurve).
* **Interacting with Nested Datasets as Columns**: Because the DataFrames are directly nested, DataFrame operations apply to the nested data intuitively. For example, filtering based on object IDs will also remove any nested photometry for the thrown out object IDs, just as any other data in the row.
* **Pyarrow-backed storage and operation**: DataFrames in DataFrames is the core idea, but not the actual implementation. Instead data is stored internally in pyarrow data structures, allowing performant storage and operation on nested datasets. You can read more about pyarrow from it's [documentation](https://arrow.apache.org/docs/python/index.html).

> **NOTE**: *In this tutorial, the terms "NestedFrame" and "DataFrame" will at times be used interchangeably. In general, these objects will always be NestedFrames but "DataFrame" is often a nice term to describe a single container of data.*

## 1. The Nested Data Representation

We can begin by walking through the basics of the NestedFrame representation. Below, we generate a toy NestedFrame which has 5 base columns("ra","dec","id","a","b"), and 1 nested column ("nested"). In a notebook, the nested column is represented as a small inner DataFrame per row, with information on the available sub-columns (sometimes referred to as fields), a preview of the first row, and the number of additional rows.

In [84]:
from lsdb.nested import generate_data

nf = generate_data(5, 10, seed=1).compute()
nf

ra        dec  id         a         b  \
0  184.255785  -8.820946  24  0.417022  0.184677   
1  342.166931  40.950381   2  0.720324  0.372520   
2   51.897461 -10.463070  36  0.000114  0.691121   
3  341.513801   5.692378  46  0.302333  0.793535   
4  112.259323 -70.888248  22  0.146756  1.077633   

                                              nested  
0  [{t: 8.38389, flux: 10.233443, band: 'g', flux...  
1  [{t: 13.70439, flux: 41.405599, band: 'g', flu...  
2  [{t: 4.089045, flux: 69.440016, band: 'g', flu...  
3  [{t: 17.562349, flux: 41.417927, band: 'g', fl...  
4  [{t: 0.547752, flux: 4.995346, band: 'r', flux...

These sub-DataFrames act as any other value stored in a DataFrame and can be accessed through normal pandas indexing:

In [85]:
nf["nested"][0]  # look at the "nested" dataframe for the first row
# or
# nf.iloc[0]["nested"]

,t,flux,band,flux_err
0,8.383890,10.233443,g,0.511672
1,13.409350,53.589641,g,2.679482
2,16.014891,90.340192,g,4.517010
3,17.892133,16.535420,g,0.826771
4,1.966937,88.330609,g,4.416530
5,6.310313,89.588622,r,4.479431
6,19.777222,11.474597,g,0.573730
7,8.957871,23.702698,g,1.185135
8,0.387339,32.664490,g,1.633225
9,1.067251,62.336012,r,3.116801


When represented lazily, nested columns are identified by their custom dtype, which also provides dtype information about the sub-columns:

In [86]:
generate_data(5, 10)

,ra,dec,id,a,b,nested
npartitions=1,,,,,,
0,float64,float64,int64,float64,float64,"nested<t: [double], flux: [double], band: [string], flux_err: [double]>"
4,...,...,...,...,...,...


Beyond just visual identification, there are programmatic ways to determine nested columns:

In [87]:
nf.nested_columns

['nested']

## 2. What We Can Do With Nested Columns

NestedFrames are pandas DataFrames, but expanded to work directly with nested columns. This is done in a few ways:
* Introduces some new access dynamics for nested columns.
* Adds several new functions specific to working with nested columns.
* Modifies some existing DataFrame functions to enable support for nested columns.


### Sub-Column Access

The first to touch on are the access dynamics, through nested sub-column access:

In [88]:
# flux is a sub-column of the "nested" column
nf["nested.flux"]

0    10.233443
0    53.589641
       ...    
4    90.853515
4    99.732285
Name: flux, Length: 50, dtype: double[pyarrow]

Using "[column].[sub-column]" allows access to the sub-column data. In the case above, it pulls out a flat pandas series of all flux values from all nested DataFrames, with appropriate index. This sub-column access pattern is available in many pandas/LSDB functions as well and augments their behavior. For example, `query`:

In [89]:
nf.query("nested.band == 'g'")

ra        dec  id         a         b  \
0  184.255785  -8.820946  24  0.417022  0.184677   
1  342.166931  40.950381   2  0.720324  0.372520   
2   51.897461 -10.463070  36  0.000114  0.691121   
3  341.513801   5.692378  46  0.302333  0.793535   
4  112.259323 -70.888248  22  0.146756  1.077633   

                                              nested  
0  [{t: 8.38389, flux: 10.233443, band: 'g', flux...  
1  [{t: 13.70439, flux: 41.405599, band: 'g', flu...  
2  [{t: 4.089045, flux: 69.440016, band: 'g', flu...  
3  [{t: 17.562349, flux: 41.417927, band: 'g', fl...  
4  [{t: 17.56285, flux: 72.599799, band: 'g', flu...

By using "nested.band" in the query string, we signal to apply the query to the nested DataFrames, in this case filtering those DataFrames by the "band" sub-column. Some operations may empty some of the sub-DataFrames, as below we apply a harsh filter on flux:

In [93]:
nf_highflux = nf.query("nested.flux > 95.")
nf_highflux

ra        dec  id         a         b  \
0  184.255785  -8.820946  24  0.417022  0.184677   
1  342.166931  40.950381   2  0.720324  0.372520   
2   51.897461 -10.463070  36  0.000114  0.691121   
3  341.513801   5.692378  46  0.302333  0.793535   
4  112.259323 -70.888248  22  0.146756  1.077633   

                                              nested  
0                                               None  
1                                               None  
2  [{t: 16.692513, flux: 96.484005, band: 'g', fl...  
3                                               None  
4  [{t: 13.995167, flux: 99.732285, band: 'g', fl...

Empty DataFrames are represented as None values, and if we wanted to reject rows that had empty DataFrames, we could do something like this:

In [94]:
nf_highflux.dropna(subset="nested")

ra        dec  id         a         b  \
2   51.897461 -10.463070  36  0.000114  0.691121   
4  112.259323 -70.888248  22  0.146756  1.077633   

                                              nested  
2  [{t: 16.692513, flux: 96.484005, band: 'g', fl...  
4  [{t: 13.995167, flux: 99.732285, band: 'g', fl...

For the most part, pandas functions that are modified are modified to allow sub-column access to apply that function to the inner dataframes. Not every function has been modified, and you can refer to [this page](https://nested-pandas.readthedocs.io/en/latest/reference/nestedframe.html#extended-pandas-dataframe-interface) to see what pandas functions have been modified so far. 

### Using Nested Columns in Analysis

Another important feature of the NestedFrame API is `reduce`, which allows columns and sub-columns to be used in external analysis functions:

In [61]:
import numpy as np

# Calculate the mean of the "flux" sub-column per row of the NestedFrame
nf.reduce(np.mean, "nested.flux")

,0
0,39.614545
1,59.695671
2,48.067207
3,54.163839
4,59.602959


We use sub-column access to signal to reduce that we're applying mean to the "flux" sub-column of the "nested" column. Let's look at a more complex example:

In [62]:
def flux_ranges(flux, band, modifier, single_band=False):
    # Calculate the flux range in a single band with a modifier
    flux = flux + modifier

    # filter to just a single band
    if single_band:
        mask = band == single_band
        flux = flux[mask]
    return {"min_flux": flux.min(), "max_flux": flux.max()}


# We use the "a" column as a modifier
nf.reduce(flux_ranges, "nested.flux", "nested.band", "a", single_band="g")

,min_flux,max_flux
0,13.751691,69.489806
1,34.205946,88.201984
2,27.645841,94.538143
3,5.900400,99.933989
4,35.103222,46.804769


`flux_ranges` is still a relatively simple function, but exemplifies some of the important aspects of `reduce`:

1. We see that we were able to pass both nested sub-columns ("nested.flux", "nested.band") as well as a base column ("a"). In this case, "a" is used to add a flat offset to the flux array. 
2. The function returns a dictionary, with string keys matched to the resulting data. This is nice to do, as it allows column names to be generated for the output. 
3. Our `flux_ranges` function has a kwarg that is not sourced from our NestedFrame. In this case, we called it at the end of the function using kwarg syntax (`single_band='g'`). Any arguments (not just kwargs) that are not sourced from the NestedFrame must be called in this way.

### The `nest` Accessor

Another NestedFrame aspect to touch on is the `.nest` accessor:

In [63]:
nf.nested.nest

This accessor object can be called on any nested column and has a small suite of additional functions used for viewing and transforming nested data. For example, we can programmatically view the available sub-columns through the accessor:

In [67]:
nf.nested.nest.fields

['t', 'flux', 'band', 'flux_err']

We can use the accessor to grab different views of our nested data:

In [69]:
# View a flat version of the nested data
# All sub-dataframes are brought to one level
nf.nested.nest.to_flat()

,t,flux,band,flux_err
0,7.582994,13.505526,g,0.675276
0,1.663783,51.087752,r,2.554388
...,...,...,...,...
4,19.752332,62.256127,r,3.112806
4,9.946964,94.962253,r,4.748113


In [71]:
# Or view the data as lists
nf.nested.nest.to_lists()

,t,flux,band,flux_err
0,[ 7.58299388 1.6637826 6.61262008 10.304683...,[13.50552616 51.0877524 84.00279781 7.065855...,['g' 'r' 'r' 'r' 'r' 'g' 'r' 'r' 'g' 'g'],[0.67527631 2.55438762 4.20013989 0.3532928 2...
1,[ 8.66628694 9.07577354 3.56606874 11.191075...,[63.51024245 36.0172414 6.17625292 92.084696...,['g' 'r' 'r' 'r' 'g' 'g' 'r' 'g' 'r' 'r'],[3.17551212 1.80086207 0.30881265 4.60423484 3...
2,[10.92518886 7.23915084 11.32085568 19.096804...,[54.63496303 29.62717253 80.67568518 35.263370...,['g' 'r' 'r' 'g' 'r' 'r' 'g' 'g' 'g' 'g'],[2.73174815 1.48135863 4.03378426 1.76316851 2...
3,[ 6.55322491 18.60044299 12.61241828 4.289500...,[79.33874289 31.88746647 86.55916167 68.334204...,['r' 'r' 'r' 'g' 'g' 'g' 'g' 'g' 'r' 'r'],[3.96693714 1.59437332 4.32795808 3.41671023 4...
4,[ 1.22906377 6.79501058 4.22847307 0.711047...,[81.93220521 83.7512767 79.6381079 22.500623...,['r' 'r' 'r' 'r' 'r' 'g' 'g' 'g' 'r' 'r'],[4.09661026 4.18756383 3.9819054 1.12503118 2...


## 3. Turning Data into Nested Columns

In many data products, the relevant data will already be packaged as nested columns. However, it's very possible your data will be provided in other formats that you would like to convert into nested columns. Let's walk through a few common examples.


### Converting from List Columns

In [65]:
import nested_pandas as npd

nf_lists = npd.NestedFrame(
    {
        "a": [10, 20, 30],
        "b": ["red", "blue", "green"],
        "c": [[1, 2, 3], [4, 5, 6], [7, 8, 9]],
        "d": [[11, 22, 33], [44, 55, 66], [77, 88, 99]],
    }
)
nf_lists

,a,b,c,d
0,10,red,"[1, 2, 3]","[11, 22, 33]"
1,20,blue,"[4, 5, 6]","[44, 55, 66]"
2,30,green,"[7, 8, 9]","[77, 88, 99]"


In this case, "c" and "d" are list columns. Because the lengths of "c" and "d" match for each row, they can be nested together. We can achieve this with the `nest_lists` function:

In [ ]:
nf_lists.nest_lists(name="nested", columns=["c", "d"])

a      b                       nested
0  10    red  [{c: 1, d: 11}; …] (3 rows)
1  20   blue  [{c: 4, d: 44}; …] (3 rows)
2  30  green  [{c: 7, d: 77}; …] (3 rows)

### Converting from Flat Tables

Another case may be that your data is available as two separate tables:

In [79]:
import nested_pandas as npd

nf_base = npd.NestedFrame({"a": [10, 20, 30], "b": ["red", "blue", "green"]})
nf_flat = npd.NestedFrame(
    {"c": [1, 2, 3, 4, 5, 6, 7, 8, 9], "d": [11, 22, 33, 44, 55, 66, 77, 88, 99]},
    index=[0, 0, 0, 1, 1, 1, 2, 2, 2],
)
nf_base

,a,b
0,10,red
1,20,blue
2,30,green


In [80]:
nf_flat

,c,d
0,1,11
0,2,22
0,3,33
1,4,44
1,5,55
1,6,66
2,7,77
2,8,88
2,9,99


In this case, nf_flat has a matched index with nf_base, and we can easily nest it using `add_nested`:

In [82]:
nf_nested = nf_base.add_nested(nf_flat, name="nested")
nf_nested

a      b                       nested
0  10    red  [{c: 1, d: 11}; …] (3 rows)
1  20   blue  [{c: 4, d: 44}; …] (3 rows)
2  30  green  [{c: 7, d: 77}; …] (3 rows)

If the index is not matched, you can set the "on" kwarg to a shared column between the two DataFrames to treat the operation more as a join.

This notebook touches on a few of the most common concepts of the NestedFrame that should be helpful for using them within LSDB workflows, where much of the API identical. For more information on NestedFrames, and nested-pandas in general, consult the [nested-pandas documentation](https://nested-pandas.readthedocs.io/en/latest/).

## About

**Authors**: Doug Branton

**Last updated on**: June 17, 2025

If you use ``lsdb`` for published research, please cite the following 
[instructions](https://docs.lsdb.io/en/stable/citation.html).